In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from redcap.project import Project
import numpy as np
import warnings
warnings.simplefilter("ignore")
from IPython.display import display, HTML
import sys
sys.path.append("/Users/golpira/Python/University of Calgary/RedCap-API-Git/RedCap_StudyLoader")
from redcap_classes_V2 import RedcapProcessor #, Record, BloodDraw
from my_functions import display_summary_tables,display_value_counts_per_study, my_tableone, analyze_hgb, plot_variables_over_time, hemoglobin_prior_to_first_rbc

### Teg-Hip
load_dotenv()
api_url = os.getenv("REDCAP_URL")
api_key_hip = os.getenv("REDCAP_TEG_HIP_API_KEY")

p1 = RedcapProcessor(api_url, api_key_hip)
df_hip_original = p1.fetch_and_process()

df_hip_demo=p1.get_all_demographics()
df_hip_lab, df_hip_teg = p1.get_all_labs()

- Data Fetched
- Columns renamed
************************************************************************************
Removing the following StudyIDs from dataset:
 - TH-226: Treated non-operatively
************************************************************************************
- Selected Eligible → enrolled patinets in Hip_Pathway Study
- Flagged patients with VTE
- Added VTE_timepoints
- Processed comorbidities and complications
- Processed timepoints
- Cleaned injury/surgery dates
- Cleaned teg and lab date/time
- Cleaned AO_OTA classification --> NEED AHDMED'S INPUT
- Added pre-operative medications
- Added UTI complications
- Flagged patients' death/Withdrawal status
- Processed fixation/arthoplasty designations
- Processed blood transfusions
- Processed CAS scores
- Removed analysis after VTE event
Records Successfully Built!
************************************************************************************
This study has 310 patients


# Hemoglonin value prior to first blood transfusion (THIS MAY NOT WORK FOR PELVIS _ THERE IS NO RBC_DATE IN PELVIS)

In [2]:
hemoglobin_prior_to_first_rbc(df_hip_lab)

,StudyID,first_rbc_time,time_injury_lab_hours,Hemoglobin_prior_to_first_transfusion,blood_date,Draw_date_lab,blood_rbc
0,TH-009,50.500000,45.666667,75,None,2018-07-08 19:10:00,0.0
1,TH-011,137.333333,100.000000,97,None,2018-07-23 10:40:00,0.0
2,TH-014,62.000000,48.000000,87,None,2018-07-31 10:00:00,0.0
3,TH-015,133.000000,6.000000,121,None,2018-07-30 17:00:00,0.0
4,TH-017,52.000000,47.000000,79,None,2018-08-07 19:00:00,0.0
5,TH-019,31.000000,9.000000,122,None,2018-08-18 02:00:00,0.0
6,TH-038,33.000000,-15.000000,103,None,2018-10-09 00:00:00,0.0
7,TH-039,60.500000,52.000000,103,None,2018-10-14 15:30:00,0.0
8,TH-057,62.250000,48.666667,77,None,2019-01-17 10:25:00,0.0
9,TH-061,61.000000,47.116667,72,None,2019-01-23 10:07:00,0.0


In [3]:
import os
import pandas as pd
from dotenv import load_dotenv
from redcap.project import Project
import numpy as np
import warnings
warnings.simplefilter("ignore")
import sys
sys.path.append("/Users/golpira/Python/University of Calgary/RedCap-API-Git/RedCap_StudyLoader")
from redcap_classes_V2 import RedcapProcessor #, Record, BloodDraw
from my_functions import display_summary_tables,display_value_counts_per_study, my_tableone, analyze_hgb, plot_variables_over_time, hemoglobin_prior_to_first_rbc

# ### Hip Pathway
load_dotenv()
api_url = os.getenv("REDCAP_URL")
api_key_pathway = os.getenv("REDCAP_HIP_PATHWAY_API_KEY")

p2 = RedcapProcessor(api_url, api_key_pathway)


df_pathway_original = p2.fetch_and_process()
df_pathway_demo=p2.get_all_demographics()
df_pathway_lab, df_pathway_teg = p2.get_all_labs()

- Data Fetched
- Columns renamed
- Selected Eligible → enrolled patinets in Hip_Pathway Study
- Flagged patients with VTE
- Added VTE_timepoints
- Processed comorbidities and complications
- Processed timepoints
- Cleaned injury/surgery dates
- Cleaned teg and lab date/time
- Cleaned AO_OTA classification --> NEED AHDMED'S INPUT
- Added pre-operative medications
- Added UTI complications
- Flagged patients' death/Withdrawal status
- Processed fixation/arthoplasty designations
- Processed blood transfusions
- Processed CAS scores
- Removed analysis after VTE event
Records Successfully Built!
************************************************************************************
This study has 43 patients


In [4]:
df_pathway_lab[(df_pathway_lab['blood_rbc_yn']=='Yes')&(df_pathway_lab['StudyID']=='HPA-014')].sort_values(by=['time_injury_lab_hours','blood_date'])

,StudyID,Study,Time,VTE_type,VTE,VTE_time,Pre_op_med,Draw_date_lab,time_injury_lab_hours,time_surgery_lab_hours,time_injury_rbc_hours,blood_rbc_yn,blood_rbc,blood_date,rbc_timepoint,Hemoglobin,Creatinine
39,HPA-014,Pathway,Admission,NaN,No,NaN,DOAC,2024-07-02 09:07:00,20.616667,-4.283333,NaN,Yes,0.0,NaN,NaN,104,65
32,HPA-014,Pathway,POD1,NaN,No,NaN,DOAC,2024-07-03 07:03:00,42.550000,17.650000,NaN,Yes,0.0,NaN,NaN,84,NaN
35,HPA-014,Pathway,POD3,NaN,No,NaN,DOAC,2024-07-05 07:00:00,90.500000,65.600000,NaN,Yes,0.0,NaN,NaN,69,NaN
34,HPA-014,Pathway,POD5,NaN,No,NaN,DOAC,2024-07-07 07:00:00,138.500000,113.600000,NaN,Yes,0.0,NaN,NaN,99,35
31,HPA-014,Pathway,POD7,NaN,No,NaN,DOAC,2024-07-09 06:18:00,185.800000,160.900000,NaN,Yes,0.0,NaN,NaN,101,52
36,HPA-014,Pathway,Week2,NaN,No,NaN,DOAC,2024-07-16 07:00:00,354.500000,329.600000,NaN,Yes,0.0,NaN,NaN,81,NaN
38,HPA-014,Pathway,Week4,NaN,No,NaN,DOAC,2024-07-29 07:00:00,666.500000,641.600000,NaN,Yes,0.0,NaN,NaN,87,NaN
37,HPA-014,Pathway,Week6,NaN,No,NaN,DOAC,2024-08-09 07:00:00,930.500000,905.600000,NaN,Yes,0.0,NaN,NaN,111,34
40,HPA-014,Pathway,Admission,NaN,No,NaN,DOAC,NaT,NaN,NaN,-12.5,Yes,4.0,2024-07-01,Admission/Baseline,NaN,NaN
33,HPA-014,Pathway,Post_Op,NaN,No,NaN,DOAC,NaT,NaN,NaN,83.5,Yes,1.0,2024-07-05,POD 3,NaN,NaN


# hemoglobin_prior_to_first_rbc

In [5]:
hemoglobin_prior_to_first_rbc(df_pathway_lab)

,StudyID,first_rbc_time,time_injury_lab_hours,Hemoglobin_prior_to_first_transfusion,blood_date,Draw_date_lab,blood_rbc
0,HPA-008,53.40,2.083333,113,None,2024-03-19 20:41:00,0.0
1,HPA-014,-12.50,NaN,NaN,NaN,NaT,NaN
2,HPA-026,40.00,2.050000,93,None,2024-10-20 10:03:00,0.0
3,HPA-028,82.00,41.000000,95,None,2024-10-31 07:00:00,0.0
4,HPA-035,31.25,15.250000,117,None,2024-12-16 08:00:00,0.0
5,HPA-050,64.75,21.300000,98,None,2025-10-04 04:33:00,0.0
6,HPA-055,85.25,69.050000,79,None,2025-12-20 07:48:00,0.0
7,HPA-059,60.50,5.683333,103,None,2026-01-08 17:11:00,0.0
8,THB-HPA-025,20.00,12.433333,108,None,2026-02-23 16:26:00,0.0


In [6]:
import os
import pandas as pd
from dotenv import load_dotenv
from redcap.project import Project
import numpy as np
import warnings
warnings.simplefilter("ignore")
import sys
sys.path.append("/Users/golpira/Python/University of Calgary/RedCap-API-Git/RedCap_StudyLoader")
from redcap_classes_V2 import RedcapProcessor #, Record, BloodDraw
from my_functions import display_summary_tables,display_value_counts_per_study, my_tableone, analyze_hgb, plot_variables_over_time, hemoglobin_prior_to_first_rbc

load_dotenv()

### Teg-Femur
api_url = os.getenv("REDCAP_URL")
api_key_femur = os.getenv("REDCAP_TEG_FEMUR_API_KEY")

p3 = RedcapProcessor(api_url, api_key_femur)



df_femur_original = p3.fetch_and_process()
df_femur_demo=p3.get_all_demographics()
df_femur_lab, df_femur_teg = p3.get_all_labs()

- Data Fetched
- Columns renamed
************************************************************************************
Removing the following StudyIDs from dataset:
 - TF-070: Multiple Surgery Patient_bilateral femur fracture
 - TF-084: Multiple Surgery Patient_bilateral femur fracture
 - TF-115: Multiple Surgery Patient_bilateral femur fracture
************************************************************************************
- Selected Eligible → enrolled patinets in Hip_Pathway Study
- Flagged patients with VTE
- Added VTE_timepoints
- Processed comorbidities and complications
- Processed timepoints
- Cleaned injury/surgery dates
- Cleaned teg and lab date/time
- Cleaned AO_OTA classification --> NEED AHDMED'S INPUT
- Added pre-operative medications
- Added UTI complications
- Flagged patients' death/Withdrawal status
- Processed fixation/arthoplasty designations
- Processed blood transfusions
- Processed CAS scores
- Removed analysis after VTE event
Records Successfully Built!
***

<div style='background:#8BD6CD; padding:15px; border-radius:0px;'>
    <span style='color:#d50000; font-size:40px; font-weight:bold;'>✖</span>
    <span style='color:#d50000; font-size:24px; font-weight:bold;'> Errors:</span>
    <div style='margin-top:10px; font-size:16px; color:#000000;'>
        - TH-171 is duplicated on POD 5.
    </div>
</div>

In [7]:
hemoglobin_prior_to_first_rbc(df_femur_lab)

,StudyID,first_rbc_time,time_injury_lab_hours,Hemoglobin_prior_to_first_transfusion,blood_date,Draw_date_lab,blood_rbc
0,TF-010,65.000000,53.950000,77,None,2018-08-12 12:57:00,0.0
1,TF-012,102.000000,40.616667,89,None,2018-09-14 10:37:00,0.0
2,TF-014,40.666667,3.650000,151,None,2018-10-09 10:59:00,0.0
3,TF-015,8.750000,NaN,NaN,NaN,NaT,NaN
4,TF-020,61.000000,NaN,NaN,NaN,NaT,NaN
5,TF-022,-7.500000,NaN,NaN,NaN,NaT,NaN
6,TF-023,5.000000,NaN,NaN,NaN,NaT,NaN
7,TF-024,-6.500000,NaN,NaN,NaN,NaT,NaN
8,TF-031,148.000000,NaN,NaN,NaN,NaT,NaN
9,TF-033,-11.333333,NaN,NaN,NaN,NaT,NaN


In [8]:
import os
import pandas as pd
from dotenv import load_dotenv
from redcap.project import Project
import numpy as np
import warnings
warnings.simplefilter("ignore")
import sys
sys.path.append("/Users/golpira/Python/University of Calgary/RedCap-API-Git/RedCap_StudyLoader")
from redcap_classes_V2 import RedcapProcessor #, Record, BloodDraw
from my_functions import display_summary_tables,display_value_counts_per_study, my_tableone, analyze_hgb, plot_variables_over_time, hemoglobin_prior_to_first_rbc
load_dotenv()

### Teg-Pelvis
api_url = os.getenv("REDCAP_URL")
api_key_pelvis = os.getenv("REDCAP_TEG_PELVIS_API_KEY")

p4 = RedcapProcessor(api_url, api_key_pelvis)

df_pelvis_original = p4.fetch_and_process()
df_pelvis_demo=p4.get_all_demographics()
df_pelvis_lab, df_pelvis_teg = p4.get_all_labs()

- Data Fetched
- Columns renamed
************************************************************************************
Removing the following StudyIDs from dataset:
 - TPA-019: Two stage surgeries related to pelvis/acetabulum - 30 April 2021
 - TPA-028: Two stage surgeries related to pelvis/acetabulum - 6th and 11th April 2021
 - TPA-035: Multiple Surgery Patient: both pelvic and femur surgery on October 19, and another pelvic surgery on October 26, 2021
 - TPA-043: Two stage surgeries related to pelvis/acetabulum - 4th and 6th April 2022
 - TPA-048: Two stage surgeries related to pelvis/acetabulum - 6th and 10th May 2022
 - TPA-079: Two stage surgeries related to pelvis/acetabulum - 6th and 12th Feb 2024
 - TPANO-001: Excluded - Non-Operative Arm
 - TPANO-002: Excluded - Non-Operative Arm
 - TPANO-003: Excluded - Non-Operative Arm
 - TPANO-004: Excluded - Non-Operative Arm
 - TPANO-005: Excluded - Non-Operative Arm
 - TPANO-006: Excluded - Non-Operative Arm
 - TPANO-007: Excluded - Non

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from redcap.project import Project
import numpy as np
import warnings
warnings.simplefilter("ignore")
from IPython.display import display, HTML
import sys
sys.path.append("/Users/golpira/Python/University of Calgary/RedCap-API-Git/RedCap_StudyLoader")
from redcap_classes_V2 import RedcapProcessor #, Record, BloodDraw
from my_functions import display_summary_tables,display_value_counts_per_study, my_tableone, analyze_hgb, plot_variables_over_time, hemoglobin_prior_to_first_rbc

### Teg-Hip
load_dotenv()
api_url = os.getenv("REDCAP_URL")
api_key_hip = os.getenv("REDCAP_HIP_ARTHOPLASTY")

p5 = RedcapProcessor(api_url, api_key_hip)
df_hip_artho_original = p5.fetch_and_process()

df_hip_artho_demo=p5.get_all_demographics()
df_hip_artho_lab, df_hip_artho_teg = p5.get_all_labs()

# Everyone was enrolled on the date  of surgery! Preop blood and enrolment days are the same!

- Data Fetched
- Columns renamed
************************************************************************************
Removing the following StudyIDs from dataset:
 - TA-042: EXCLUDED
************************************************************************************
- Selected Eligible → enrolled patinets in Hip_Pathway Study
- Flagged patients with VTE
- Added VTE_timepoints
- Processed comorbidities and complications
- Processed timepoints
- Cleaned injury/surgery dates
- Cleaned teg and lab date/time
- Cleaned AO_OTA classification --> NEED AHDMED'S INPUT
- Added pre-operative medications
- Added UTI complications
- Flagged patients' death/Withdrawal status
- Processed fixation/arthoplasty designations
- Processed blood transfusions
- Processed CAS scores
- Removed analysis after VTE event
Records Successfully Built!
************************************************************************************
This study has 112 patients


In [2]:
df_hip_artho_lab.to_excel('arthro_lab.xlsx', index=False)

In [3]:
df_hip_artho_teg.to_excel('arthro_teg.xlsx', index=False)

In [8]:
df_hip_artho_teg[df_hip_artho_teg['VTE']=='Yes'][['StudyID','Time','VTE_time','R_time', 'K_time','Alpha_Angle', 'MA','LY30', 'ACT','ADP-agg', 'ADP-inh','ADP-ma','AA-agg','AA-inh','AA-ma','CFF-MA','ACTF-MA','CFF-A10','CFF-FLEV']]

,StudyID,Time,VTE_time,R_time,K_time,Alpha_Angle,MA,LY30,ACT,ADP-agg,ADP-inh,ADP-ma,AA-agg,AA-inh,AA-ma,CFF-MA,ACTF-MA,CFF-A10,CFF-FLEV
91,TA-015,Week6,Week10,0.4,1.3,75.8,62.9,0.0,87.9,92.4,7.6,43.6,86.1,13.9,41.5,19.9,13.1,NaN,NaN
92,TA-015,Week4,Week10,0.2,1.4,75.5,62.4,0.0,69.2,87.1,12.9,55.0,63.4,36.6,44.0,21.1,14.6,NaN,NaN
93,TA-015,Week2,Week10,0.3,0.8,80.4,70.7,0.0,78.5,NaN,NaN,NaN,NaN,NaN,NaN,40.5,NaN,37.8,NaN
94,TA-015,POD2,Week10,0.3,0.9,81.3,70.2,0.1,78.5,67.5,32.5,52.5,21.3,78.7,33.9,41.1,25.3,37.6,NaN
95,TA-015,Unscheduled,Week10,0.5,1.1,82.0,73.8,0.0,97.3,99.5,0.5,71.4,100.0,0.0,71.8,43.8,31.8,42.2,NaN
96,TA-015,Post_Op,Week10,0.2,1.0,77.5,64.6,0.0,69.2,88.7,11.3,58.3,94.6,5.4,61.0,21.2,17.4,NaN,NaN
97,TA-015,Pre_Op,Week10,0.4,1.3,78.3,62.3,0.0,87.9,94.8,5.2,61.1,94.6,5.4,61.0,20.0,17.4,NaN,NaN
98,TA-015,POD1,Week10,0.5,1.0,76.3,64.1,0.2,97.3,87.1,12.9,58.4,62.3,37.7,46.9,22.2,18.0,NaN,NaN
342,TA-056,NaN,POD2,0.4,3.1,78.5,68.8,NaN,87.9,76.8,23.2,58.4,34.4,65.6,39.8,34.6,24.7,30.5,NaN
343,TA-056,NaN,POD2,0.7,2.4,76.8,67,0,116,9.3,90.7,20.6,97.9,2.1,63.6,25.2,16.1,23.8,NaN


In [4]:
df_hip_artho_demo.to_excel('arthro_demo.xlsx', index=False)

In [13]:
ALL_DEMO=pd.concat([df_hip_demo,df_pathway_demo,df_femur_demo,df_pelvis_demo,df_hip_artho_demo])
ALL_DEMO.to_excel('ALL_TEG_DEMO.xlsx', index=False)

In [14]:
ALL_LAB=pd.concat([df_hip_lab,df_pathway_lab,df_femur_lab,df_pelvis_lab,df_hip_artho_lab])
ALL_TEG=pd.concat([df_hip_teg,df_pathway_teg,df_femur_teg,df_pelvis_teg,df_hip_artho_teg])


ALL_LAB = ALL_LAB[ALL_LAB['Time']!='Unscheduled']
ALL_TEG = ALL_TEG[ALL_TEG['Time']!='Unscheduled']

In [15]:
ALL_TEG[ALL_TEG.duplicated(subset=['StudyID','Time'], keep=False)][['StudyID','Time','Alpha_Angle','Draw_date_teg','time_injury_teg_hours']]

,StudyID,Time,Alpha_Angle,Draw_date_teg,time_injury_teg_hours
1402,TH-171,POD5,79.1,2022-01-14 07:00:00,4669.000000
1403,TH-171,POD5,77.9,2021-07-09 06:52:00,132.866667
120,TA-019,NaN,80.6,2022-11-08 09:45:00,NaN
123,TA-019,NaN,74.2,NaT,NaN
134,TA-021,NaN,77.0,2023-01-05 12:35:00,NaN
...,...,...,...,...,...
720,TA-113,NaN,NaN,NaT,NaN
721,TA-113,NaN,NaN,NaT,NaN
722,TA-113,NaN,NaN,NaT,NaN
723,TA-113,NaN,NaN,NaT,NaN


In [16]:
ALL_TEG[ALL_TEG['StudyID']=='TH-094'][['StudyID','Time','Alpha_Angle','Draw_date_teg','time_injury_teg_hours']]

,StudyID,Time,Alpha_Angle,Draw_date_teg,time_injury_teg_hours
788,TH-094,POD4,78.7,2019-08-07 09:53:00,128.883333
789,TH-094,POD2,77.6,2019-08-05 19:07:00,90.116667
790,TH-094,POD3,79.8,2019-08-06 08:15:00,103.250000
791,TH-094,POD5,79.2,2019-08-08 01:00:00,144.000000
792,TH-094,POD1,77.2,2019-08-04 08:19:00,55.316667
793,TH-094,PFD1,77.8,2019-08-03 06:43:00,29.716667
794,TH-094,Month3,78.3,2019-11-18 09:58:00,2600.966667
795,TH-094,Admission,74.9,2019-08-02 09:18:00,8.300000
796,TH-094,Week2,80.2,2019-08-13 08:45:00,271.750000
797,TH-094,Week4,77.6,2019-09-03 01:20:00,768.333333


In [17]:
ALL_TEG[ALL_TEG['StudyID']=='TPA-061'][['VTE_time','Time', 'MA',"VTE"]]

,VTE_time,Time,MA,VTE
406,POD9,POD3,71.5,Yes
407,POD9,POD1,68.3,Yes
408,POD9,PFD3,66.5,Yes
409,POD9,PFD2,69.2,Yes
410,POD9,POD7,71.6,Yes
411,POD9,POD5,72.6,Yes


In [18]:
ALL_LAB[ALL_LAB['blood_rbc_yn']=='Yes'][['Study','StudyID','Time','rbc_timepoint','blood_rbc','Hemoglobin','blood_date','blood_rbc_yn','Draw_date_lab','time_injury_lab_hours','time_injury_rbc_hours']].head(60)

,Study,StudyID,Time,rbc_timepoint,blood_rbc,Hemoglobin,blood_date,blood_rbc_yn,Draw_date_lab,time_injury_lab_hours,time_injury_rbc_hours
34,Hip,TH-009,POD2,NaN,0.0,75,NaN,Yes,2018-07-08 19:10:00,45.666667,NaN
35,Hip,TH-009,POD3,NaN,0.0,60,NaN,Yes,2018-07-09 20:45:00,71.250000,NaN
36,Hip,TH-009,POD4,NaN,0.0,76,NaN,Yes,2018-07-10 20:00:00,94.500000,NaN
37,Hip,TH-009,POD1,NaN,0.0,NaN,NaN,Yes,2018-07-07 00:00:00,2.500000,NaN
38,Hip,TH-009,Admission,NaN,0.0,95,NaN,Yes,2018-07-07 00:00:00,2.500000,NaN
39,Hip,TH-009,Post_Op,Post-Operative,1.0,NaN,2018-07-09,Yes,NaT,NaN,50.500000
40,Hip,TH-011,POD3,NaN,0.0,NaN,NaN,Yes,2018-07-24 00:00:00,113.333333,NaN
41,Hip,TH-011,POD2,NaN,0.0,97,NaN,Yes,2018-07-23 10:40:00,100.000000,NaN
42,Hip,TH-011,POD1,NaN,0.0,103,NaN,Yes,2018-07-22 12:44:00,78.066667,NaN
43,Hip,TH-011,Admission,NaN,0.0,134,NaN,Yes,2018-07-19 16:25:00,9.750000,NaN


In [19]:
ALL_LAB[ALL_LAB['blood_rbc_yn']=='Yes'].sort_values(['StudyID']).head(20)

,StudyID,Study,Time,VTE_type,VTE,VTE_time,Pre_op_med,Draw_date_lab,time_injury_lab_hours,time_surgery_lab_hours,time_injury_rbc_hours,blood_rbc_yn,blood_rbc,blood_date,rbc_timepoint,Hemoglobin,Creatinine
8,HPA-008,Pathway,POD1,NaN,No,NaN,DOAC,2024-03-22 06:08:00,59.533333,21.633333,NaN,Yes,0.0,NaN,NaN,99,NaN
14,HPA-008,Pathway,Admission,NaN,No,NaN,DOAC,2024-03-19 20:41:00,2.083333,-35.816667,NaN,Yes,0.0,NaN,NaN,113,65
13,HPA-008,Pathway,POD5,NaN,No,NaN,DOAC,2024-03-26 07:00:00,156.400000,118.500000,NaN,Yes,0.0,NaN,NaN,95,NaN
9,HPA-008,Pathway,POD3,NaN,No,NaN,DOAC,2024-03-24 08:00:00,109.400000,71.500000,NaN,Yes,0.0,NaN,NaN,91,NaN
11,HPA-008,Pathway,POD7,NaN,No,NaN,DOAC,2024-03-28 07:00:00,204.400000,166.500000,NaN,Yes,0.0,NaN,NaN,96,NaN
10,HPA-008,Pathway,Post_Op,NaN,No,NaN,DOAC,NaT,NaN,NaN,77.4,Yes,1.0,2024-03-23,POD 2,NaN,NaN
12,HPA-008,Pathway,Post_Op,NaN,No,NaN,DOAC,NaT,NaN,NaN,53.4,Yes,2.0,2024-03-22,POD 1,NaN,NaN
39,HPA-014,Pathway,Admission,NaN,No,NaN,DOAC,2024-07-02 09:07:00,20.616667,-4.283333,NaN,Yes,0.0,NaN,NaN,104,65
38,HPA-014,Pathway,Week4,NaN,No,NaN,DOAC,2024-07-29 07:00:00,666.500000,641.600000,NaN,Yes,0.0,NaN,NaN,87,NaN
37,HPA-014,Pathway,Week6,NaN,No,NaN,DOAC,2024-08-09 07:00:00,930.500000,905.600000,NaN,Yes,0.0,NaN,NaN,111,34


In [20]:
ALL_LAB[ALL_LAB.duplicated(subset=['StudyID','Time'], keep=False)] [['StudyID','Time','Hemoglobin','blood_rbc','Draw_date_lab','time_injury_lab_hours','blood_date','rbc_timepoint']]

# .to_excel('Test.xlsx', index=False)


#

,StudyID,Time,Hemoglobin,blood_rbc,Draw_date_lab,time_injury_lab_hours,blood_date,rbc_timepoint
59,TH-014,Post_Op,68,0.0,2018-07-30 00:00:00,14.000000,NaN,NaN
61,TH-014,Post_Op,NaN,2.0,NaT,NaN,2018-08-01,Post-Operative
846,TH-171,POD5,78,0.0,2022-01-14 07:00:00,4669.000000,NaN,NaN
847,TH-171,POD5,103,0.0,2021-07-09 06:52:00,132.866667,NaN,NaN
1161,TH-258,Post_Op,83,0.0,2023-02-07 00:00:00,13.500000,NaN,NaN
...,...,...,...,...,...,...,...,...
5,TPA-003,Pre_Op,106,0.0,2020-02-26 04:00:00,37.500000,NaN,NaN
10,TPA-003,Pre_Op,NaN,3.0,NaT,NaN,NaN,Admission to Pre-Operative
278,TPA-073,Pre_Op,98,0.0,NaT,NaN,NaN,NaN
279,TPA-073,Pre_Op,104,0.0,2023-09-10 17:03:00,7.050000,NaN,NaN


In [21]:
ALL_TEG=pd.merge(ALL_DEMO,ALL_TEG, on=['StudyID','Study','VTE','VTE_type','VTE_time','Pre_op_med'], how='outer', indicator=True)
ALL_LAB=pd.merge(ALL_DEMO,ALL_LAB, on=['StudyID','Study','VTE','VTE_type','VTE_time','Pre_op_med','blood_rbc_yn'], how='outer', indicator=True)


ALL_TEG.to_excel('ALL_TEG.xlsx', index=False)
ALL_LAB.to_excel('ALL_LAB.xlsx', index=False)


In [22]:
ALL_TEG[ALL_TEG.duplicated(subset=['StudyID','Time'], keep=False)][['StudyID','Time','Alpha_Angle','Draw_date_teg','time_injury_teg_hours']]

,StudyID,Time,Alpha_Angle,Draw_date_teg,time_injury_teg_hours
366,TA-019,NaN,80.6,2022-11-08 09:45:00,NaN
369,TA-019,NaN,74.2,NaT,NaN
380,TA-021,NaN,77.0,2023-01-05 12:35:00,NaN
381,TA-021,Week6,75.6,NaT,NaN
382,TA-021,NaN,71.1,NaT,NaN
...,...,...,...,...,...
944,TA-113,NaN,NaN,NaT,NaN
945,TA-113,NaN,NaN,NaT,NaN
946,TA-113,NaN,NaN,NaT,NaN
3222,TH-171,POD5,79.1,2022-01-14 07:00:00,4669.000000


In [23]:
ALL_DEMO['Study'].value_counts()

Study
Hip            310
Femur          134
Arthoplasty    112
Pelvis          98
Pathway         43
Name: count, dtype: int64

In [24]:
display_summary_tables(ALL_DEMO, filter_col='VTE', extra_col='VTE_type')


No,StudyID,VTE_type
1,TH-003,DVT
2,TH-082,PE
3,TH-088,PE
4,TH-201,DVT
5,TH-227,DVT
6,TH-253,DVT
7,TH-261,Both
8,TH-264,DVT
9,TH-271,PE
10,TH-279,Both


In [25]:
ALL_DEMO[ALL_DEMO['VTE']=='Yes'][['StudyID','VTE_type', 'VTE_time']]

,StudyID,VTE_type,VTE_time
2,TH-003,DVT,Week6
81,TH-082,PE,Week2
87,TH-088,PE,POD1
200,TH-201,DVT,POD2
225,TH-227,DVT,POD3
251,TH-253,DVT,POD2
259,TH-261,Both,POD3
262,TH-264,DVT,POD3
269,TH-271,PE,POD2
277,TH-279,Both,POD2


In [26]:
display_summary_tables(ALL_DEMO, filter_col='Death')

No,StudyID
1,TH-023
2,TH-030
3,TH-069
4,TH-082
5,TH-088
6,TH-100
7,TH-130
8,TH-133
9,TH-206
10,TH-267


In [27]:
display_summary_tables(ALL_DEMO, filter_col='Withdrawn')

No,StudyID
1,TH-004
2,TH-010
3,TH-011
4,TH-015
5,TH-019
6,TH-020
7,TH-022
8,TH-027
9,TH-031
10,TH-037


In [28]:
display_value_counts_per_study(ALL_DEMO, 'VTE')

No,VTE,count
1,No,298
2,Yes,12
No,VTE,count
1,No,127
2,Yes,7
No,VTE,count
1,No,84
2,Yes,14
No,VTE,count
1,No,42


In [29]:
display_value_counts_per_study(ALL_DEMO, 'Pre_op_med')

No,Pre_op_med,count
1,LMWH,255
2,ASA,34
3,DOAC,16
4,No Info,4
5,Warfarin,1
No,Pre_op_med,count
1,LMWH,126
2,ASA,7
3,DOAC,1
No,Pre_op_med,count


In [30]:
display_value_counts_per_study(ALL_DEMO, 'Sex')

No,Sex,count
1,Female,201
2,Male,109
No,Sex,count
1,Male,85
2,Female,49
No,Sex,count
1,Male,70
2,Female,28
No,Sex,count
1,Female,25


In [31]:
display_value_counts_per_study(ALL_DEMO, 'AO_OTA')

No,AO_OTA,count
1,31-A3,98
2,31-B1,54
3,31-B3,53
4,31-B2,48
5,31-A2,33
6,31-A1,24
No,AO_OTA,count
1,31-A3,40
2,32-A,38
3,33-A,17


In [32]:
display_summary_tables(ALL_DEMO, filter_col='Pre_op_med')

No,StudyID
No,StudyID
No,StudyID
No,StudyID
No,StudyID


In [33]:
display_summary_tables(ALL_DEMO, filter_col='blood_rbc_yn')

No,StudyID
1,TH-009
2,TH-011
3,TH-014
4,TH-015
5,TH-017
6,TH-019
7,TH-038
8,TH-039
9,TH-057
10,TH-061


In [34]:
display_value_counts_per_study(ALL_DEMO, 'blood_rbc_yn')

No,blood_rbc_yn,count
1,No,257
2,Yes,53
No,blood_rbc_yn,count
1,No,86
2,Yes,48
No,blood_rbc_yn,count
1,No,59
2,Yes,39
No,blood_rbc_yn,count
1,No,34


In [35]:
display(ALL_LAB[ALL_LAB['time_injury_lab_hours']<0][['StudyID','Time','Injury_date','Draw_date_lab','time_injury_lab_hours']])

,StudyID,Time,Injury_date,Draw_date_lab,time_injury_lab_hours
375,TF-007,Post_Op,2018-08-04 03:50,2018-08-04 00:09:00,-3.683333
560,TF-056,Admission,2021-06-22 12:00,2021-06-22 10:18:00,-1.700000
903,TH-007,Admission,2018-07-05 18:45,2018-07-05 00:00:00,-18.750000
907,TH-008,Admission,2018-07-06 06:00,2018-07-06 00:00:00,-6.000000
973,TH-021,Admission,2018-08-21 15:30,2018-08-21 00:00:00,-15.500000
999,TH-027,Admission,2018-09-07 05:30,2018-09-07 00:00:00,-5.500000
1024,TH-032,Admission,2018-09-20 10:00,2018-09-20 00:00:00,-10.000000
1057,TH-038,Admission,2018-10-09 15:00,2018-10-09 00:00:00,-15.000000
1065,TH-039,Admission,2018-10-12 11:30,2018-10-12 00:00:00,-11.500000
1076,TH-040,Admission,2018-10-24 04:30,2018-10-24 00:00:00,-4.500000


In [36]:
display(ALL_TEG[ALL_TEG['time_injury_teg_hours']<0][['StudyID','Time','Injury_date','Draw_date_teg','time_injury_teg_hours']])

,StudyID,Time,Injury_date,Draw_date_teg,time_injury_teg_hours
980,TF-007,Post_Op,2018-08-04 03:50,2018-08-04 00:09:00,-3.683333
1277,TF-056,Admission,2021-06-22 12:00,2021-06-22 10:18:00,-1.700000
2008,TH-023,Admission,2018-08-31 19:15,2018-08-31 06:45:00,-12.500000
2527,TH-083,Admission,2019-06-16 16:00,2019-06-16 10:15:00,-5.750000
3168,TH-163,Admission,2021-06-15 22:30,2021-06-15 17:15:00,-5.250000


In [37]:
df_demo=pd.concat([df_hip_demo,df_pathway_demo]).reset_index(drop=True)


df_demo_oac = df_demo[df_demo['Pre_op_med'].isin(['DOAC','Warfarin']) ]
df_blood_draws = pd.concat([p1.get_all_blood_draws(), p2.get_all_blood_draws()]).drop_duplicates().reset_index(drop=True)

AttributeError: 'RedcapProcessor' object has no attribute 'get_all_blood_draws'

# ONLY DOAC - Hemoglobin drop

In [ ]:
df_blood_draws_oac = df_blood_draws[df_blood_draws['StudyID'].isin(df_demo_oac['StudyID'])]

In [ ]:
df_blood_draws_oac

In [ ]:
analyze_hgb(df_blood_draws_oac, pod_time="POD1")

In [ ]:
analyze_hgb(df_blood_draws_oac, pod_time="POD3")

In [ ]:
# df_blood_draws[df_blood_draws['Time']=='Week2'].sort_values('StudyID')

In [ ]:
df_blood_draws.head()

In [ ]:
df_blood_draws[df_blood_draws['Time'].isin(['Admission','Pre-Op','PFD1','Pre-Op','PFD2','4 hr Pre-OP','Pre-draw','PFD3','PFD4'])].head(10)

## Created two columns to pick depending on if we want pre-op closest to injury or if we want pre-op closest to surgery

In [ ]:
preop_timepoints = ['Admission','Pre-Op','PFD1','Pre-Op','PFD2','4 hr Pre-OP','Pre-draw','PFD3','PFD4']

# Initialize new columns
df_blood_draws['closest_to_injury'] = 'No'
df_blood_draws['closest_to_surgery'] = 'No'


columns_to_check = ['R_time', 'K_time', 'MA', 'LY30', 'ACT','Alpha_Angle']

for patient_id, group in df_blood_draws.groupby('StudyID'):
    preop_draws = group[group['Time'].isin(preop_timepoints)].copy()
    
    if not preop_draws.empty:
        # Filter out draws where all columns are NaN
        valid_draws = preop_draws.dropna(subset=columns_to_check, how='all')
        
        if not valid_draws.empty:
            # Closest to injury
            closest_idx = valid_draws['injury_to_lab_hrs'].idxmin()
            df_blood_draws.loc[closest_idx, 'closest_to_injury'] = 'Yes'
            
            # Closest to surgery
            farthest_idx = valid_draws['injury_to_lab_hrs'].idxmax()
            df_blood_draws.loc[farthest_idx, 'closest_to_surgery'] = 'Yes'
        else:
            # If all draws are invalid, fallback to first/last pre-op draw
            df_blood_draws.loc[preop_draws.index[0], 'closest_to_injury'] = 'Yes'
            df_blood_draws.loc[preop_draws.index[-1], 'closest_to_surgery'] = 'Yes'
            
# # Loop over each patient
# for patient_id, group in df_blood_draws.groupby('StudyID'):
#     preop_draws = group[group['Time'].isin(preop_timepoints)].copy()
    
#     if not preop_draws.empty:
#         # Closest to injury
#         if preop_draws['time_from_injury_to_draw_hours'].notna().any():
#             closest_idx = preop_draws['time_from_injury_to_draw_hours'].idxmin()
#         elif (preop_draws['Time'] == 'Admission').any():
#             closest_idx = preop_draws[preop_draws['Time'] == 'Admission'].index[0]
#         else:
#             # Fallback: just take the first pre-op draw
#             closest_idx = preop_draws.index[0]
#         df_blood_draws.loc[closest_idx, 'closest_to_injury'] = 'Yes'
        
#         # Closest to surgery (farthest from injury)
#         if preop_draws['time_from_injury_to_draw_hours'].notna().any():
#             farthest_idx = preop_draws['time_from_injury_to_draw_hours'].idxmax()
#         elif (preop_draws['Time'] == 'Admission').any():
#             farthest_idx = preop_draws[preop_draws['Time'] == 'Admission'].index[0]
#         else:
#             # Fallback: just take the last pre-op draw
#             farthest_idx = preop_draws.index[-1]
#         df_blood_draws.loc[farthest_idx, 'closest_to_surgery'] = 'Yes'

# Update 'Time' for closest to injury
df_blood_draws['Time'] = np.where(
    df_blood_draws['closest_to_injury'] == 'Yes',
    'Pre_op',
    df_blood_draws['Time'])

In [ ]:
# df_blood_draws[df_blood_draws['StudyID']=='TH-137']

In [ ]:
df_blood_draws[df_blood_draws['closest_to_injury']=='Yes'].head()

In [ ]:
columns = ['Sex', 'Age', 'BMI','VTE_type','VTE']
groupby = 'Pre_op_med'
categorical = ['Sex','VTE_type','VTE']

nonnormal=['BMI'] 
display(TableOne(df_demo, columns=columns, categorical=categorical, nonnormal=nonnormal, groupby=groupby))

In [ ]:
columns = ['R_time', 'K_time',  'MA', 'LY30', 'ACT','Alpha_Angle'] 
groupby = 'Pre_op_med'
categorical = []


for timepoint in ['Pre_op','POD1', 'POD3', 'POD5', 'POD7', 'Week2', 'Week4', 'Week6', 'Month3']:
    print(timepoint)
    print('++++++++++++++++++++++++++++++++++++++++++++')
    df=df_blood_draws[df_blood_draws['Time']==timepoint]


    

    nonnormal=[ 'R_time', 'K_time',  'MA', 'LY30', 'ACT','Alpha_Angle'] 
    display(TableOne(df, columns=columns, categorical=categorical, nonnormal=nonnormal, groupby=groupby))

In [ ]:
df_blood_draws.columns

In [ ]:
for col in columns:
    df_blood_draws[col] = pd.to_numeric(df_blood_draws[col], errors='coerce')

    
plot_variables_over_time(df_blood_draws,
                         hue='VTE', 
    style='VTE',
    palette={'Yes':"#ff3636", 'No':"#030303"},
    # var_labels=var_labels,
    xlabel="Timepoints",
)

NameError: name 'columns' is not defined

In [ ]:
df_blood_draws[(df_blood_draws['StudyID']=='TH-190')&(df_blood_draws['Time']=='Pre_op')][['Time','LY30','Pre_op_med']]

In [ ]:
display(all_blood_hip[all_blood_hip['StudyID']=='TH-188'][['Time','LY30']])